# Train Network

This notebook trains a Convolutional Neural Network using categorized images.

### Install Packages

This notebook requires `Pillow`, `tensorflow` and `keras`. You may uncomment and run the cell below to have Jupyter Notebook install these for you.

In [1]:
# !pip install Pillow, tensorflow, keras

### Configuration

Set the variables below to specify where the categorical images are located, which classes to train and what the filename should be for saved training data. If you would like to train all 45+ categories, set `classes = None`.

In [38]:
import os

image_path = 'capture_data'
epochs = 100

classes = [ "forward", "slight_left", "left", "sharp_left", "slight_right", "right", "sharp_right" ]
checkpoint_path = 'weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'

In [39]:
import matplotlib.image as mpimg
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import Image
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(classes), activation='softmax'))
    return model

In [40]:
def train():
    from keras.preprocessing.image import ImageDataGenerator
    from keras.callbacks import ModelCheckpoint

    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=1)

    datagen = ImageDataGenerator(rescale = 1.0 / 255.0,  
                                 rotation_range=10, width_shift_range=0.2, height_shift_range=0.2,
                                 zoom_range=0.2,shear_range=20,
                                 validation_split=0.2)

    training_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                               color_mode='grayscale', batch_size=32, shuffle=True,
                                               subset="training")
    validation_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                               color_mode='grayscale', batch_size=32, shuffle=True,
                                               subset="validation")

    history = model.fit_generator(training_set,
                                  validation_data=validation_set,
                                  epochs=epochs, verbose=1, callbacks=[ checkpoint ])
    
train()

Found 174 images belonging to 7 classes.
Found 40 images belonging to 7 classes.
Epoch 1/100
6/6 [==============================] - 5s 761ms/step - loss: 2.0118 - acc: 0.3286 - val_loss: 1.8026 - val_acc: 0.4250

Epoch 00001: val_acc improved from -inf to 0.42500, saving model to weights-improvement-01-0.42.hdf5
Epoch 2/100
6/6 [==============================] - 4s 598ms/step - loss: 1.8632 - acc: 0.4092 - val_loss: 1.8699 - val_acc: 0.4250

Epoch 00002: val_acc did not improve
Epoch 3/100
6/6 [==============================] - 4s 638ms/step - loss: 1.8177 - acc: 0.3828 - val_loss: 1.6965 - val_acc: 0.4250

Epoch 00003: val_acc did not improve
Epoch 4/100
6/6 [==============================] - 4s 606ms/step - loss: 1.7213 - acc: 0.3876 - val_loss: 1.6261 - val_acc: 0.4500

Epoch 00004: val_acc improved from 0.42500 to 0.45000, saving model to weights-improvement-04-0.45.hdf5
Epoch 5/100
6/6 [==============================] - 4s 619ms/step - loss: 1.6401 - acc: 0.4370 - val_loss: 1.5069

6/6 [==============================] - 4s 611ms/step - loss: 0.9215 - acc: 0.6119 - val_loss: 1.0489 - val_acc: 0.5500

Epoch 00046: val_acc did not improve
Epoch 47/100
6/6 [==============================] - 4s 616ms/step - loss: 0.8967 - acc: 0.6542 - val_loss: 1.1483 - val_acc: 0.5750

Epoch 00047: val_acc did not improve
Epoch 48/100
6/6 [==============================] - 4s 608ms/step - loss: 0.9642 - acc: 0.5855 - val_loss: 1.0176 - val_acc: 0.6000

Epoch 00048: val_acc did not improve
Epoch 49/100
6/6 [==============================] - 4s 615ms/step - loss: 0.8215 - acc: 0.7084 - val_loss: 1.1616 - val_acc: 0.5500

Epoch 00049: val_acc did not improve
Epoch 50/100
6/6 [==============================] - 4s 605ms/step - loss: 0.9957 - acc: 0.6009 - val_loss: 1.0404 - val_acc: 0.5500

Epoch 00050: val_acc did not improve
Epoch 51/100
6/6 [==============================] - 4s 609ms/step - loss: 0.8655 - acc: 0.6397 - val_loss: 1.0403 - val_acc: 0.5250

Epoch 00051: val_acc did not i

6/6 [==============================] - 4s 629ms/step - loss: 0.7275 - acc: 0.6863 - val_loss: 0.9971 - val_acc: 0.6250

Epoch 00094: val_acc did not improve
Epoch 95/100
6/6 [==============================] - 4s 662ms/step - loss: 0.7636 - acc: 0.6749 - val_loss: 0.9869 - val_acc: 0.6000

Epoch 00095: val_acc did not improve
Epoch 96/100
6/6 [==============================] - 4s 688ms/step - loss: 0.7516 - acc: 0.6820 - val_loss: 1.1299 - val_acc: 0.6000

Epoch 00096: val_acc did not improve
Epoch 97/100
6/6 [==============================] - 5s 762ms/step - loss: 0.7767 - acc: 0.6427 - val_loss: 1.0985 - val_acc: 0.5750

Epoch 00097: val_acc did not improve
Epoch 98/100
6/6 [==============================] - 4s 667ms/step - loss: 0.7743 - acc: 0.6586 - val_loss: 0.9912 - val_acc: 0.6500

Epoch 00098: val_acc did not improve
Epoch 99/100
6/6 [==============================] - 4s 740ms/step - loss: 0.7760 - acc: 0.7286 - val_loss: 1.1837 - val_acc: 0.6500

Epoch 00099: val_acc did not i

In [41]:
def load_trained_model(weightfile):
    from keras.models import load_model
    model = load_model(weightfile)
    return model

In [42]:
trained_model = load_trained_model("sample_weights.hdf5")


In [45]:
import cv2
import numpy as np
image = cv2.imread("capture_data/forward/Groot1529091688.png", 0)
image = np.array([ image.reshape(64, 64, 1) ])
print(classes[trained_model.predict_classes(image)[0]])

['forward', 'slight_left', 'left', 'sharp_left', 'slight_right', 'right', 'sharp_right']
left
